# Pre-work TikTok soundcharts API

## Setup

In [ ]:
# Import modules
import pandas as pd

from datetime import datetime, timedelta
from soundcharts_API_client import *
from tqdm import tqdm


In [ ]:
# API connexion
x_app_id = 'PARIS1_E22B0B40'
x_api_key = '3c7bfb46ee2705df'

sdch = SoundchartsClient(x_app_id, x_api_key)


## Get data

### Get track and artist-level infos

In [ ]:
# Import playlist data
df = pd.read_excel('tiktok_charts_tracks_with_use.xlsx')


In [ ]:
# Get the tracks' and artists' info
results_df = []

for index, row in df.iterrows():
    track_id = row['track_uuid']
    
    collection_date = row['collection_date']
    track_position = row['track_position']
    artist_video = row['artist_video']
    artist_video_date = row['artist_video_date']

    tracks = sdch.get_track_metadata(track_id=track_id)

    track_names = tracks.get("object", {}).get("name")
    tracks_ids = tracks.get("object", {}).get("uuid")
    track_dates = tracks.get("object", {}).get("releaseDate")
    track_features = tracks.get("object", {}).get("audio")

    artist_names = []
    artist_ids = []

    for artist in tracks.get("object", {}).get("artists", []):
        artist_names.append(artist.get("name"))
        artist_ids.append(artist.get("uuid"))

    label_names = []
    label_types = []

    for label in tracks.get("object", {}).get("labels", []):
        label_names.append(label.get("name"))
        label_types.append(label.get("type"))
    
    track_genres = []
    
    for genre in tracks.get("object", {}).get("genres", []):
        track_genres.append(genre.get("root"))

    collection_date_dt = datetime.fromisoformat(collection_date)
    date_norm = collection_date_dt.strftime('%Y-%m-%d')

    followers = []

    for artist_id in artist_ids:
        artists_followers = sdch.get_artist_tiktok_followers_by_date(
            artist_id=artist_id,
            startDate=(collection_date_dt - timedelta(days=1)).strftime('%Y-%m-%d'),
            endDate=date_norm
        )

        if 'items' in artists_followers and artists_followers['items']:
            follower_count = artists_followers['items'][0].get('followerCount', None)
            followers.append(follower_count)
        else:
            followers.append(None)

    results_df.append({
        "collection_date": collection_date,
        "track_position": track_position,
        "track_name": track_names,
        "track_id": tracks_ids,
        "artist_name": artist_names,
        "artist_id": artist_ids,
        "artist_followers": followers,
        "label_name": label_names,
        "label_type": label_types,
        "track_date": track_dates,
        "track_genres": track_genres,
        "track_features": track_features,
        "artist_video": artist_video,
        "artist_video_date": artist_video_date
    })

# Convert the list to a DataFrame
df = pd.DataFrame(results_df)
df.to_csv('tiktok_tracks_info.csv', index=False)

# Print the DataFrame
display(df)


### Get tracks' streams and tiktok videos

In [ ]:
# Import tracks playlist data
df = pd.read_excel('tiktok_tracks_info.csv')


In [ ]:
# Get the streams for each track
results_df = []

for index, row in df.iterrows():
    collection_date = row['collection_date']
    track_id = row['track_id']
    track_name = row['track_name']
    track_position = row['track_position']
    track_date = row['track_date']
    track_genres = row['track_genres']
    track_features = row['track_features']
    artist_name = row['artist_name']
    artist_id = row['artist_id']
    artis_followers = row['artist_followers']
    label_name = row['label_name']
    label_type = row['label_type']
    artist_video = row['artist_video']
    artist_video_date = row['artist_video_date']

    collection_date_dt = datetime.fromisoformat(collection_date)
    date_norm = collection_date_dt.strftime('%Y-%m-%d')

    streams_data = sdch.get_track_streams_by_date(
        startDate=date_norm,
        endDate=datetime.now().strftime('%Y-%m-%d'),
        track_id=track_id
    )

    processed_date = set()
    
    for item in streams_data.get("items", []):
        date = item.get("date")
        
        if date in processed_date:
            continue
        
        processed_date.add(date)
        
        first_plot = next(iter(item.get("plots", [])), None)
        
        if first_plot:
            results_df.append({
                "collection_date": collection_date,
                "track_position": track_position,
                "track_name": track_name,
                "track_id": track_id,
                "artist_name": artist_name,
                "artist_id": artist_id,
                "label_name": label_name,
                "label_type": label_type,
                "track_date": track_date,
                "track_genres": track_genres,
                "track_features": track_features,
                "stream_date": date,
                "streams": first_plot.get("value")
            })

# Convert the list to a DataFrame
df = pd.DataFrame(results_df)
df.to_excel('tracks_info_streams.xlsx', index=False)

# Print the DataFrame
display(df)
